In [21]:
# ========================================================================
# FORCE MATPLOTLIB INLINE (PUT THIS FIRST!)
# ========================================================================
%matplotlib inline

import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Force matplotlib to use Agg backend for Jupyter
import matplotlib
matplotlib.use('Agg')
plt.ioff()  # Turn off interactive mode

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['figure.dpi'] = 100  # Set DPI for clarity

print("=" * 70)
print("📊 HOTEL REVENUE ANALYTICS - KEY INSIGHTS")
print("=" * 70)
print("\n✅ Matplotlib configured for inline display")
print(f"   Backend: {matplotlib.get_backend()}")

📊 HOTEL REVENUE ANALYTICS - KEY INSIGHTS

✅ Matplotlib configured for inline display
   Backend: Agg


In [22]:
"""
========================================================================
HOTEL REVENUE ANALYTICS - KEY BUSINESS INSIGHTS
========================================================================
Skills: SQL Queries, Data Analysis, Business Intelligence, Visualization
========================================================================
"""

import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 6)

print("=" * 70)
print("📊 HOTEL REVENUE ANALYTICS - KEY INSIGHTS")
print("=" * 70)
print("\n🎯 Goal: Extract actionable business insights from data")

📊 HOTEL REVENUE ANALYTICS - KEY INSIGHTS

🎯 Goal: Extract actionable business insights from data


In [23]:
# ========================================================================
# CONNECT TO DATABASE
# ========================================================================
print("\n📁 Connecting to database...")

conn = sqlite3.connect('../data/hotel_revenue.db')

# Verify connection
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
print(f"✅ Connected successfully!")
print(f"   Available tables: {', '.join(tables['name'].tolist())}")


📁 Connecting to database...
✅ Connected successfully!
   Available tables: dim_date, dim_room, sqlite_sequence, dim_segment, fact_bookings


In [24]:
# ========================================================================
# INSIGHT 1: OVERALL BUSINESS PERFORMANCE
# ========================================================================
print("\n💼 INSIGHT 1: Overall Business Performance")
print("=" * 70)

query = """
SELECT 
    COUNT(*) as total_bookings,
    SUM(CASE WHEN is_cancelled = 0 THEN 1 ELSE 0 END) as completed,
    SUM(CASE WHEN is_cancelled = 1 THEN 1 ELSE 0 END) as cancelled,
    ROUND(SUM(revenue), 2) as total_revenue,
    ROUND(AVG(CASE WHEN is_cancelled = 0 THEN adr END), 2) as avg_adr,
    ROUND(AVG(CASE WHEN is_cancelled = 0 THEN nights END), 2) as avg_nights,
    ROUND(AVG(CASE WHEN is_cancelled = 0 THEN guests END), 2) as avg_guests
FROM fact_bookings
"""

metrics = pd.read_sql(query, conn)

# Display metrics
print(f"\n📊 KEY METRICS:")
print(f"   Total Bookings: {metrics.iloc[0]['total_bookings']:,}")
print(f"   Completed: {metrics.iloc[0]['completed']:,} ({metrics.iloc[0]['completed']/metrics.iloc[0]['total_bookings']*100:.1f}%)")
print(f"   Cancelled: {metrics.iloc[0]['cancelled']:,} ({metrics.iloc[0]['cancelled']/metrics.iloc[0]['total_bookings']*100:.1f}%)")
print(f"\n💰 REVENUE METRICS:")
print(f"   Total Revenue: €{metrics.iloc[0]['total_revenue']:,.2f}")
print(f"   Average ADR: €{metrics.iloc[0]['avg_adr']:.2f}")
print(f"   Average Length of Stay: {metrics.iloc[0]['avg_nights']:.1f} nights")
print(f"   Average Guests per Booking: {metrics.iloc[0]['avg_guests']:.1f}")

# Calculate additional metrics
avg_booking_value = metrics.iloc[0]['total_revenue'] / metrics.iloc[0]['completed']
print(f"   Average Booking Value: €{avg_booking_value:.2f}")


💼 INSIGHT 1: Overall Business Performance

📊 KEY METRICS:
   Total Bookings: 97,391.0
   Completed: 61,310.0 (63.0%)
   Cancelled: 36,081.0 (37.0%)

💰 REVENUE METRICS:
   Total Revenue: €21,484,255.10
   Average ADR: €102.53
   Average Length of Stay: 3.4 nights
   Average Guests per Booking: 2.0
   Average Booking Value: €350.42


In [25]:
# ========================================================================
# INSIGHT 2: BEST PERFORMING ROOM TYPES
# ========================================================================
print("\n🛏️  INSIGHT 2: Best Performing Room Types")
print("=" * 70)

query = """
SELECT 
    r.room_type,
    COUNT(f.booking_key) as bookings,
    ROUND(SUM(f.revenue), 2) as revenue,
    ROUND(AVG(f.adr), 2) as avg_adr,
    ROUND(AVG(f.nights), 2) as avg_nights,
    ROUND(r.base_price, 2) as base_price
FROM fact_bookings f
JOIN dim_room r ON f.room_key = r.room_key
WHERE f.is_cancelled = 0
GROUP BY r.room_type, r.base_price
ORDER BY revenue DESC
"""

rooms = pd.read_sql(query, conn)

print("\n📊 Room Type Performance:")
print(rooms.to_string(index=False))

# Calculate revenue contribution
rooms['revenue_pct'] = (rooms['revenue'] / rooms['revenue'].sum() * 100).round(1)

print(f"\n💡 KEY INSIGHTS:")
print(f"   Top Revenue Room: {rooms.iloc[0]['room_type']} (€{rooms.iloc[0]['revenue']:,.2f}, {rooms.iloc[0]['revenue_pct']:.1f}%)")
print(f"   Most Booked Room: {rooms.loc[rooms['bookings'].idxmax(), 'room_type']} ({rooms['bookings'].max():,} bookings)")
print(f"   Highest ADR Room: {rooms.loc[rooms['avg_adr'].idxmax(), 'room_type']} (€{rooms['avg_adr'].max():.2f})")


🛏️  INSIGHT 2: Best Performing Room Types

📊 Room Type Performance:
room_type  bookings     revenue  avg_adr  avg_nights  base_price
        A     41940 11745609.37    90.88        3.07       90.00
        D     11294  5229112.98   119.44        4.02      123.30
        E      3871  2045045.11   120.46        4.47      115.00
        F      1700   987631.59   163.38        3.57      179.10
        G      1139   755664.93   172.61        3.72      179.00
        C       490   357162.80   158.39        4.34      168.00
        H       300   191201.41   183.63        3.25      185.00
        B       576   172826.91    90.84        3.28       90.46

💡 KEY INSIGHTS:
   Top Revenue Room: A (€11,745,609.37, 54.7%)
   Most Booked Room: A (41,940 bookings)
   Highest ADR Room: H (€183.63)


In [26]:
# ========================================================================
# INSIGHT 3: SEASONAL REVENUE PATTERNS
# ========================================================================
print("\n📅 INSIGHT 3: Seasonal Revenue Patterns")
print("=" * 70)

query = """
SELECT 
    d.month,
    d.month_name,
    ROUND(SUM(f.revenue), 2) as revenue,
    COUNT(f.booking_key) as bookings,
    ROUND(AVG(f.adr), 2) as avg_adr
FROM fact_bookings f
JOIN dim_date d ON f.date_key = d.date_key
WHERE f.is_cancelled = 0
GROUP BY d.month, d.month_name
ORDER BY d.month
"""

monthly = pd.read_sql(query, conn)

print("\n📊 Monthly Revenue Performance:")
print(monthly.to_string(index=False))

# Find peak and low seasons
peak_month = monthly.loc[monthly['revenue'].idxmax()]
low_month = monthly.loc[monthly['revenue'].idxmin()]

print(f"\n💡 SEASONAL INSIGHTS:")
print(f"   Peak Month: {peak_month['month_name']} (€{peak_month['revenue']:,.2f})")
print(f"   Low Month: {low_month['month_name']} (€{low_month['revenue']:,.2f})")
print(f"   Seasonal Variance: {((peak_month['revenue'] / low_month['revenue']) - 1) * 100:.1f}% difference")

# Calculate quarterly performance
query_quarterly = """
SELECT 
    d.year,
    d.quarter,
    ROUND(SUM(f.revenue), 2) as revenue,
    COUNT(f.booking_key) as bookings
FROM fact_bookings f
JOIN dim_date d ON f.date_key = d.date_key
WHERE f.is_cancelled = 0
GROUP BY d.year, d.quarter
ORDER BY d.year, d.quarter
"""

quarterly = pd.read_sql(query_quarterly, conn)

print(f"\n📊 Quarterly Performance:")
print(quarterly.to_string(index=False))


📅 INSIGHT 3: Seasonal Revenue Patterns

📊 Monthly Revenue Performance:
 month month_name    revenue  bookings  avg_adr
     1    January  776676.84      4122    67.00
     2   February 1147541.48      5372    72.42
     3      March 1652162.39      6644    77.34
     4      April 2099857.48      6565    97.88
     5        May 2394703.64      7113   104.79
     6       June 2583290.92      6404   114.59
     7       July 3342057.28      6402   133.91
     8     August 3779506.35      6347   152.12
     9  September 1289642.69      3372   115.29
    10    October 1072101.84      3689    93.25
    11   November  688843.47      2818    78.24
    12   December  657870.72      2462    84.98

💡 SEASONAL INSIGHTS:
   Peak Month: August (€3,779,506.35)
   Low Month: December (€657,870.72)
   Seasonal Variance: 474.5% difference

📊 Quarterly Performance:
 year  quarter    revenue  bookings
 2016        1 1516029.52      7592
 2016        2 3114669.39     10126
 2016        3 4623986.49      96

In [27]:
# ========================================================================
# INSIGHT 4: TOP MARKET SEGMENTS
# ========================================================================
print("\n👥 INSIGHT 4: Top Market Segments")
print("=" * 70)

query = """
SELECT 
    s.market_segment,
    COUNT(f.booking_key) as bookings,
    ROUND(SUM(f.revenue), 2) as revenue,
    ROUND(AVG(f.adr), 2) as avg_adr,
    ROUND(AVG(f.nights), 2) as avg_nights,
    ROUND(AVG(f.guests), 2) as avg_guests
FROM fact_bookings f
JOIN dim_segment s ON f.segment_key = s.segment_key
WHERE f.is_cancelled = 0
GROUP BY s.market_segment
ORDER BY revenue DESC
"""

segments = pd.read_sql(query, conn)

# Calculate contribution percentages
segments['revenue_pct'] = (segments['revenue'] / segments['revenue'].sum() * 100).round(1)
segments['booking_pct'] = (segments['bookings'] / segments['bookings'].sum() * 100).round(1)

print("\n📊 Market Segment Performance:")
print(segments.to_string(index=False))

print(f"\n💡 SEGMENT INSIGHTS:")
print(f"   Top Revenue Segment: {segments.iloc[0]['market_segment']} (€{segments.iloc[0]['revenue']:,.2f}, {segments.iloc[0]['revenue_pct']}%)")
print(f"   Most Frequent Segment: {segments.loc[segments['bookings'].idxmax(), 'market_segment']} ({segments['bookings'].max():,} bookings)")
print(f"   Highest Spending Segment: {segments.loc[segments['avg_adr'].idxmax(), 'market_segment']} (€{segments['avg_adr'].max():.2f} ADR)")


👥 INSIGHT 4: Top Market Segments

📊 Market Segment Performance:
market_segment  bookings     revenue  avg_adr  avg_nights  avg_guests  revenue_pct  booking_pct
     Online TA     31218 12023118.84   115.56        3.31        2.10         56.0         50.9
 Offline TA/TO     11895  4302157.09    84.10        4.33        1.88         20.0         19.4
        Direct      8709  3291609.26   115.28        3.01        2.06         15.3         14.2
        Groups      5456  1365693.04    77.91        3.35        1.69          6.4          8.9
     Corporate      3351   426872.98    68.42        1.81        1.19          2.0          5.5
      Aviation       185    70868.36    99.48        3.74        1.01          0.3          0.3
 Complementary       496     3935.53     3.23        1.65        1.61          0.0          0.8

💡 SEGMENT INSIGHTS:
   Top Revenue Segment: Online TA (€12,023,118.84, 56.0%)
   Most Frequent Segment: Online TA (31,218 bookings)
   Highest Spending Segment: Onlin

In [28]:
# ========================================================================
# INSIGHT 5: WEEKEND VS WEEKDAY PERFORMANCE
# ========================================================================
print("\n📆 INSIGHT 5: Weekend vs Weekday Performance")
print("=" * 70)

query = """
SELECT 
    CASE WHEN d.is_weekend = 1 THEN 'Weekend' ELSE 'Weekday' END as day_type,
    COUNT(f.booking_key) as bookings,
    ROUND(SUM(f.revenue), 2) as revenue,
    ROUND(AVG(f.adr), 2) as avg_adr,
    ROUND(AVG(f.nights), 2) as avg_nights
FROM fact_bookings f
JOIN dim_date d ON f.date_key = d.date_key
WHERE f.is_cancelled = 0
GROUP BY d.is_weekend
"""

weekend = pd.read_sql(query, conn)

print("\n📊 Weekend vs Weekday Comparison:")
print(weekend.to_string(index=False))

# Calculate premium
weekend_adr = weekend[weekend['day_type'] == 'Weekend']['avg_adr'].values[0]
weekday_adr = weekend[weekend['day_type'] == 'Weekday']['avg_adr'].values[0]
weekend_premium = ((weekend_adr / weekday_adr) - 1) * 100

print(f"\n💡 WEEKEND INSIGHTS:")
print(f"   Weekend Premium: {weekend_premium:+.1f}%")
if weekend_premium > 0:
    print(f"   → Weekend rates are {weekend_premium:.1f}% higher than weekdays")
else:
    print(f"   → Weekday rates are {abs(weekend_premium):.1f}% higher than weekends")


📆 INSIGHT 5: Weekend vs Weekday Performance

📊 Weekend vs Weekday Comparison:
day_type  bookings     revenue  avg_adr  avg_nights
 Weekday     44526 15091115.97   101.93        3.26
 Weekend     16784  6393139.13   104.11        3.68

💡 WEEKEND INSIGHTS:
   Weekend Premium: +2.1%
   → Weekend rates are 2.1% higher than weekdays


In [29]:
# ========================================================================
# INSIGHT 6: CANCELLATION ANALYSIS
# ========================================================================
print("\n🚫 INSIGHT 6: Cancellation Patterns")
print("=" * 70)

# Overall cancellation rate
query = """
SELECT 
    COUNT(*) as total,
    SUM(CASE WHEN is_cancelled = 1 THEN 1 ELSE 0 END) as cancelled,
    ROUND(AVG(CASE WHEN is_cancelled = 1 THEN 1.0 ELSE 0.0 END) * 100, 2) as cancel_rate
FROM fact_bookings
"""

cancel_overall = pd.read_sql(query, conn)

print(f"\n📊 Overall Cancellation Rate: {cancel_overall.iloc[0]['cancel_rate']:.1f}%")
print(f"   Total Bookings: {cancel_overall.iloc[0]['total']:,}")
print(f"   Cancelled: {cancel_overall.iloc[0]['cancelled']:,}")
print(f"   Completed: {cancel_overall.iloc[0]['total'] - cancel_overall.iloc[0]['cancelled']:,}")

# Cancellation by segment
query = """
SELECT 
    s.market_segment,
    COUNT(*) as total_bookings,
    SUM(CASE WHEN f.is_cancelled = 1 THEN 1 ELSE 0 END) as cancelled,
    ROUND(AVG(CASE WHEN f.is_cancelled = 1 THEN 1.0 ELSE 0.0 END) * 100, 2) as cancel_rate
FROM fact_bookings f
JOIN dim_segment s ON f.segment_key = s.segment_key
GROUP BY s.market_segment
ORDER BY cancel_rate DESC
"""

cancel_segment = pd.read_sql(query, conn)

print(f"\n📊 Cancellation Rate by Segment:")
print(cancel_segment.to_string(index=False))

print(f"\n💡 CANCELLATION INSIGHTS:")
print(f"   Highest Cancel Rate: {cancel_segment.iloc[0]['market_segment']} ({cancel_segment.iloc[0]['cancel_rate']}%)")
print(f"   Lowest Cancel Rate: {cancel_segment.iloc[-1]['market_segment']} ({cancel_segment.iloc[-1]['cancel_rate']}%)")

# Calculate lost revenue
query = """
SELECT 
    ROUND(SUM(CASE WHEN is_cancelled = 1 THEN adr * nights ELSE 0 END), 2) as lost_revenue
FROM fact_bookings
"""

lost_rev = pd.read_sql(query, conn)
print(f"\n💰 Potential Lost Revenue from Cancellations: €{lost_rev.iloc[0]['lost_revenue']:,.2f}")


🚫 INSIGHT 6: Cancellation Patterns

📊 Overall Cancellation Rate: 37.0%
   Total Bookings: 97,391.0
   Cancelled: 36,081.0
   Completed: 61,310.0

📊 Cancellation Rate by Segment:
market_segment  total_bookings  cancelled  cancel_rate
        Groups           13710       8254        60.20
     Online TA           50312      19094        37.95
 Offline TA/TO           18138       6243        34.42
      Aviation             237         52        21.94
     Corporate            4124        773        18.74
        Direct           10292       1583        15.38
 Complementary             578         82        14.19

💡 CANCELLATION INSIGHTS:
   Highest Cancel Rate: Groups (60.2%)
   Lowest Cancel Rate: Complementary (14.19%)

💰 Potential Lost Revenue from Cancellations: €14,415,279.67


In [30]:
# ========================================================================
# CHART 1: REVENUE BY ROOM TYPE
# ========================================================================
print("\n📊 CHART 1: Revenue by Room Type")
print("=" * 70)

# Create figure
plt.figure(figsize=(10, 6))

# Create bars
plt.barh(rooms['room_type'], rooms['revenue'], 
         color='steelblue', alpha=0.7, edgecolor='black', linewidth=2)

# Labels and title
plt.title('Revenue by Room Type', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Revenue (€)', fontsize=12, fontweight='bold')
plt.ylabel('Room Type', fontsize=12, fontweight='bold')

# Add value labels
for i, (room, rev) in enumerate(zip(rooms['room_type'], rooms['revenue'])):
    plt.text(rev, i, f'  €{rev/1000:.0f}K', 
            va='center', fontweight='bold', fontsize=11)

plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()

# CRITICAL: Display the plot
plt.show()

print("✅ Chart 1 complete")
print(f"   Top performer: {rooms.iloc[0]['room_type']} (€{rooms.iloc[0]['revenue']:,.0f})")


📊 CHART 1: Revenue by Room Type
✅ Chart 1 complete
   Top performer: A (€11,745,609)


In [31]:
# ========================================================================
# CHART 2: MONTHLY REVENUE TREND
# ========================================================================
print("\n📊 CHART 2: Monthly Revenue Trend")
print("=" * 70)

# Create figure
plt.figure(figsize=(12, 6))

# Line plot
plt.plot(monthly['month'], monthly['revenue'], 
        marker='o', linewidth=3, markersize=12, 
        color='#2ecc71', markerfacecolor='#2ecc71', 
        markeredgecolor='black', markeredgewidth=2)

# Fill area
plt.fill_between(monthly['month'], 0, monthly['revenue'], 
                alpha=0.3, color='#2ecc71')

# Labels
plt.title('Monthly Revenue Trend', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Month', fontsize=12, fontweight='bold')
plt.ylabel('Revenue (€)', fontsize=12, fontweight='bold')

# X-axis
plt.xticks(range(1, 13), 
          ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
           'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], 
          rotation=45)

# Highlight peak
peak_idx = monthly['revenue'].idxmax()
plt.scatter(monthly.loc[peak_idx, 'month'], 
           monthly.loc[peak_idx, 'revenue'], 
           s=500, color='red', zorder=5, 
           edgecolors='black', linewidths=3, marker='*')

plt.grid(True, alpha=0.3)
plt.tight_layout()

# CRITICAL: Display the plot
plt.show()

print("✅ Chart 2 complete")
peak_month_name = monthly.loc[peak_idx, 'month_name']
print(f"   Peak month: {peak_month_name} (€{monthly.loc[peak_idx, 'revenue']:,.0f})")


📊 CHART 2: Monthly Revenue Trend
✅ Chart 2 complete
   Peak month: August (€3,779,506)


In [32]:
# ========================================================================
# CHART 3: REVENUE BY SEGMENT
# ========================================================================
print("\n📊 CHART 3: Revenue Share by Market Segment")
print("=" * 70)

# Create figure
plt.figure(figsize=(10, 8))

# Sort by revenue
segments_sorted = segments.sort_values('revenue', ascending=False)

# Create pie chart
plt.pie(segments_sorted['revenue'], 
       labels=segments_sorted['market_segment'],
       autopct='%1.1f%%',
       startangle=90,
       colors=plt.cm.Set3(range(len(segments_sorted))),
       textprops={'fontsize': 11, 'fontweight': 'bold'},
       wedgeprops={'edgecolor': 'black', 'linewidth': 2})

plt.title('Revenue Share by Market Segment', 
         fontsize=16, fontweight='bold', pad=20)

plt.tight_layout()

# CRITICAL: Display the plot
plt.show()

print("✅ Chart 3 complete")
print(f"   Top segment: {segments_sorted.iloc[0]['market_segment']} "
      f"({segments_sorted.iloc[0]['revenue_pct']:.1f}%)")


📊 CHART 3: Revenue Share by Market Segment
✅ Chart 3 complete
   Top segment: Online TA (56.0%)


In [33]:
# ========================================================================
# CHART 4: WEEKEND VS WEEKDAY
# ========================================================================
print("\n📊 CHART 4: Weekend vs Weekday Comparison")
print("=" * 70)

# Create figure
plt.figure(figsize=(10, 6))

# Get data
weekday_row = weekend[weekend['day_type'] == 'Weekday'].iloc[0]
weekend_row = weekend[weekend['day_type'] == 'Weekend'].iloc[0]

# Prepare data
categories = ['Bookings', 'Revenue', 'ADR']
weekday_vals = [weekday_row['bookings'], weekday_row['revenue'], weekday_row['avg_adr']]
weekend_vals = [weekend_row['bookings'], weekend_row['revenue'], weekend_row['avg_adr']]

# Bar positions
x = [0, 1, 2]
width = 0.35

# Create bars
plt.bar([i - width/2 for i in x], weekday_vals, width, 
       label='Weekday', color='#3498db', alpha=0.8, 
       edgecolor='black', linewidth=2)

plt.bar([i + width/2 for i in x], weekend_vals, width, 
       label='Weekend', color='#e74c3c', alpha=0.8, 
       edgecolor='black', linewidth=2)

# Labels
plt.title('Weekend vs Weekday Performance', 
         fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Value', fontsize=12, fontweight='bold')
plt.xticks(x, categories, fontsize=11, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()

# CRITICAL: Display the plot
plt.show()

print("✅ Chart 4 complete")

# Calculate premium
weekend_premium = ((weekend_row['avg_adr'] / weekday_row['avg_adr']) - 1) * 100
print(f"   Weekend premium: {weekend_premium:+.1f}%")


📊 CHART 4: Weekend vs Weekday Comparison
✅ Chart 4 complete
   Weekend premium: +2.1%


In [34]:
# ========================================================================
# ADVANCED ANALYSIS
# ========================================================================
print("\n🔬 ADVANCED ANALYSIS: Room Type × Market Segment")
print("=" * 70)

query = """
SELECT 
    r.room_type,
    s.market_segment,
    COUNT(f.booking_key) as bookings,
    ROUND(SUM(f.revenue), 2) as revenue,
    ROUND(AVG(f.adr), 2) as avg_adr
FROM fact_bookings f
JOIN dim_room r ON f.room_key = r.room_key
JOIN dim_segment s ON f.segment_key = s.segment_key
WHERE f.is_cancelled = 0
GROUP BY r.room_type, s.market_segment
ORDER BY revenue DESC
LIMIT 10
"""

matrix = pd.read_sql(query, conn)

print("\n📊 Top 10 Room-Segment Combinations (by Revenue):")
print(matrix.to_string(index=False))

# Create pivot table for heatmap
query_pivot = """
SELECT 
    r.room_type,
    s.market_segment,
    ROUND(SUM(f.revenue), 0) as revenue
FROM fact_bookings f
JOIN dim_room r ON f.room_key = r.room_key
JOIN dim_segment s ON f.segment_key = s.segment_key
WHERE f.is_cancelled = 0
GROUP BY r.room_type, s.market_segment
"""

pivot_data = pd.read_sql(query_pivot, conn)
pivot_table = pivot_data.pivot(index='room_type', columns='market_segment', values='revenue')
pivot_table = pivot_table.fillna(0)

print(f"\n💡 COMBINATION INSIGHTS:")
print(f"   Best Combination: {matrix.iloc[0]['room_type']} × {matrix.iloc[0]['market_segment']}")
print(f"   Revenue: €{matrix.iloc[0]['revenue']:,.2f}")
print(f"   Bookings: {matrix.iloc[0]['bookings']:,}")


🔬 ADVANCED ANALYSIS: Room Type × Market Segment

📊 Top 10 Room-Segment Combinations (by Revenue):
room_type market_segment  bookings    revenue  avg_adr
        A      Online TA     19757 6221760.01   102.28
        D      Online TA      7110 3346444.76   131.26
        A  Offline TA/TO      9310 2812605.16    82.00
        A         Direct      4671 1199436.04    95.67
        A         Groups      4763 1120899.95    77.81
        D  Offline TA/TO      1838  986978.39    87.26
        E      Online TA      1838  959320.94   128.08
        D         Direct      1584  676270.87   124.78
        F      Online TA      1009  580687.57   179.41
        E         Direct      1034  532403.27   133.82

💡 COMBINATION INSIGHTS:
   Best Combination: A × Online TA
   Revenue: €6,221,760.01
   Bookings: 19,757


In [35]:
# Create heatmap
fig, ax = plt.subplots(figsize=(12, 6))

sns.heatmap(pivot_table, annot=True, fmt='.0f', cmap='YlGnBu', 
           linewidths=0.5, ax=ax, cbar_kws={'label': 'Revenue (€)'})

ax.set_title('Revenue Heatmap: Room Type × Market Segment', 
            fontsize=14, fontweight='bold', pad=15)
ax.set_xlabel('Market Segment', fontsize=12)
ax.set_ylabel('Room Type', fontsize=12)

plt.tight_layout()
plt.show()

print("✅ Heatmap created!")

✅ Heatmap created!


In [36]:
# ========================================================================
# KEY BUSINESS RECOMMENDATIONS
# ========================================================================
print("\n" + "=" * 70)
print("💡 KEY BUSINESS RECOMMENDATIONS")
print("=" * 70)

print(f"""
Based on the analysis, here are actionable recommendations:

1️⃣ ROOM STRATEGY
   Focus: {rooms.iloc[0]['room_type']} (Top revenue: €{rooms.iloc[0]['revenue']:,.0f})
   Action: 
   • Ensure maximum availability for this room type
   • Consider premium pricing during peak season
   • Prioritize maintenance and upgrades
   
2️⃣ SEASONAL PLANNING
   Peak: {peak_month['month_name']} (€{peak_month['revenue']:,.0f})
   Low: {low_month['month_name']} (€{low_month['revenue']:,.0f})
   Action:
   • Increase prices by 10-15% during {peak_month['month_name']}
   • Run promotional campaigns in {low_month['month_name']}
   • Adjust staffing levels by season
   
3️⃣ MARKET SEGMENT FOCUS
   Priority: {segments.iloc[0]['market_segment']} (€{segments.iloc[0]['revenue']:,.0f}, {segments.iloc[0]['revenue_pct']}%)
   Action:
   • Strengthen partnerships with this segment
   • Tailor marketing campaigns
   • Offer loyalty programs
   
4️⃣ WEEKEND PRICING
   Premium: {weekend_premium:+.1f}%
   Action:
   • {"Continue weekend premium pricing" if weekend_premium > 0 else "Consider implementing weekend premium"}
   • Target weekend leisure travelers
   • Create weekend packages
   
5️⃣ CANCELLATION MANAGEMENT
   Rate: {cancel_overall.iloc[0]['cancel_rate']:.1f}%
   Lost Revenue: €{lost_rev.iloc[0]['lost_revenue']:,.0f}
   Action:
   • Implement stricter cancellation policies
   • Offer non-refundable discounts
   • Focus on segments with low cancellation rates
   
6️⃣ REVENUE OPPORTUNITY
   Best Combo: {matrix.iloc[0]['room_type']} × {matrix.iloc[0]['market_segment']}
   Action:
   • Create targeted packages for this combination
   • Optimize pricing strategy
   • Focus sales efforts here
""")


💡 KEY BUSINESS RECOMMENDATIONS

Based on the analysis, here are actionable recommendations:

1️⃣ ROOM STRATEGY
   Focus: A (Top revenue: €11,745,609)
   Action: 
   • Ensure maximum availability for this room type
   • Consider premium pricing during peak season
   • Prioritize maintenance and upgrades
   
2️⃣ SEASONAL PLANNING
   Peak: August (€3,779,506)
   Low: December (€657,871)
   Action:
   • Increase prices by 10-15% during August
   • Run promotional campaigns in December
   • Adjust staffing levels by season
   
3️⃣ MARKET SEGMENT FOCUS
   Priority: Online TA (€12,023,119, 56.0%)
   Action:
   • Strengthen partnerships with this segment
   • Tailor marketing campaigns
   • Offer loyalty programs
   
4️⃣ WEEKEND PRICING
   Premium: +2.1%
   Action:
   • Continue weekend premium pricing
   • Target weekend leisure travelers
   • Create weekend packages
   
5️⃣ CANCELLATION MANAGEMENT
   Rate: 37.0%
   Lost Revenue: €14,415,280
   Action:
   • Implement stricter cancellation po

In [37]:
# ========================================================================
# EXPORT KEY INSIGHTS
# ========================================================================
print("\n💾 Exporting key insights to CSV...")
print("=" * 70)

# Export room performance
rooms.to_csv('../data/room_performance.csv', index=False)
print("✅ Exported: room_performance.csv")

# Export monthly trends
monthly.to_csv('../data/monthly_trends.csv', index=False)
print("✅ Exported: monthly_trends.csv")

# Export segment analysis
segments.to_csv('../data/segment_analysis.csv', index=False)
print("✅ Exported: segment_analysis.csv")

# Export cancellation analysis
cancel_segment.to_csv('../data/cancellation_analysis.csv', index=False)
print("✅ Exported: cancellation_analysis.csv")

# Create summary report
summary = {
    'Metric': [
        'Total Bookings',
        'Completed Bookings',
        'Cancellation Rate',
        'Total Revenue',
        'Average ADR',
        'Average Nights',
        'Peak Month',
        'Top Room Type',
        'Top Segment',
        'Weekend Premium'
    ],
    'Value': [
        f"{metrics.iloc[0]['total_bookings']:,}",
        f"{metrics.iloc[0]['completed']:,}",
        f"{cancel_overall.iloc[0]['cancel_rate']:.1f}%",
        f"€{metrics.iloc[0]['total_revenue']:,.2f}",
        f"€{metrics.iloc[0]['avg_adr']:.2f}",
        f"{metrics.iloc[0]['avg_nights']:.1f}",
        peak_month['month_name'],
        rooms.iloc[0]['room_type'],
        segments.iloc[0]['market_segment'],
        f"{weekend_premium:+.1f}%"
    ]
}

summary_df = pd.DataFrame(summary)
summary_df.to_csv('../data/executive_summary.csv', index=False)
print("✅ Exported: executive_summary.csv")

print(f"\n📁 All insights exported to /data folder")


💾 Exporting key insights to CSV...
✅ Exported: room_performance.csv
✅ Exported: monthly_trends.csv
✅ Exported: segment_analysis.csv
✅ Exported: cancellation_analysis.csv
✅ Exported: executive_summary.csv

📁 All insights exported to /data folder


In [38]:
# ========================================================================
# FINAL SUMMARY
# ========================================================================
print("\n" + "=" * 70)
print("🎉 ANALYSIS COMPLETE!")
print("=" * 70)

print(f"""
✅ WHAT WE DISCOVERED:

📊 Business Performance:
   • {metrics.iloc[0]['completed']:,} completed bookings
   • €{metrics.iloc[0]['total_revenue']:,.2f} total revenue
   • {cancel_overall.iloc[0]['cancel_rate']:.1f}% cancellation rate

🛏️  Room Insights:
   • Best performer: {rooms.iloc[0]['room_type']}
   • Highest ADR: {rooms.loc[rooms['avg_adr'].idxmax(), 'room_type']}
   • {len(rooms)} room types analyzed

📅 Seasonal Patterns:
   • Peak: {peak_month['month_name']} (€{peak_month['revenue']:,.0f})
   • Low: {low_month['month_name']} (€{low_month['revenue']:,.0f})
   • {((peak_month['revenue'] / low_month['revenue']) - 1) * 100:.0f}% seasonal variance

👥 Market Segments:
   • Top: {segments.iloc[0]['market_segment']} ({segments.iloc[0]['revenue_pct']}%)
   • {len(segments)} segments analyzed
   • Clear differentiation in spending patterns

💡 KEY RECOMMENDATIONS:
   1. Focus on {rooms.iloc[0]['room_type']} room type
   2. Optimize pricing for {peak_month['month_name']}
   3. Target {segments.iloc[0]['market_segment']} segment
   4. Reduce cancellations (save €{lost_rev.iloc[0]['lost_revenue']:,.0f})
   
📁 EXPORTED FILES:
   • room_performance.csv
   • monthly_trends.csv
   • segment_analysis.csv
   • cancellation_analysis.csv
   • executive_summary.csv

🎯 NEXT STEPS:
   Run Notebook 4: Revenue Forecasting
""")

# Close connection
conn.close()
print("\n✅ Database connection closed")
print("=" * 70)


🎉 ANALYSIS COMPLETE!

✅ WHAT WE DISCOVERED:

📊 Business Performance:
   • 61,310.0 completed bookings
   • €21,484,255.10 total revenue
   • 37.0% cancellation rate

🛏️  Room Insights:
   • Best performer: A
   • Highest ADR: H
   • 8 room types analyzed

📅 Seasonal Patterns:
   • Peak: August (€3,779,506)
   • Low: December (€657,871)
   • 475% seasonal variance

👥 Market Segments:
   • Top: Online TA (56.0%)
   • 7 segments analyzed
   • Clear differentiation in spending patterns

💡 KEY RECOMMENDATIONS:
   1. Focus on A room type
   2. Optimize pricing for August
   3. Target Online TA segment
   4. Reduce cancellations (save €14,415,280)
   
📁 EXPORTED FILES:
   • room_performance.csv
   • monthly_trends.csv
   • segment_analysis.csv
   • cancellation_analysis.csv
   • executive_summary.csv

🎯 NEXT STEPS:
   Run Notebook 4: Revenue Forecasting


✅ Database connection closed


In [39]:
# ========================================================================
# SAVE CHARTS AS IMAGES (FALLBACK SOLUTION)
# ========================================================================
print("\n💾 Saving charts as image files...")
print("=" * 70)

import os

# Create output directory
os.makedirs('../outputs', exist_ok=True)

# -------------------------
# Chart 1: Revenue by Room
# -------------------------
plt.figure(figsize=(10, 6))
plt.barh(rooms['room_type'], rooms['revenue'], 
         color='steelblue', alpha=0.7, edgecolor='black', linewidth=2)
plt.title('Revenue by Room Type', fontsize=16, fontweight='bold')
plt.xlabel('Revenue (€)', fontsize=12)
plt.ylabel('Room Type', fontsize=12)
for i, (room, rev) in enumerate(zip(rooms['room_type'], rooms['revenue'])):
    plt.text(rev, i, f'  €{rev/1000:.0f}K', va='center', fontweight='bold')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig('../outputs/chart1_revenue_by_room.png', dpi=150, bbox_inches='tight')
plt.close()
print("✅ Saved: chart1_revenue_by_room.png")

# -------------------------
# Chart 2: Monthly Trend
# -------------------------
plt.figure(figsize=(12, 6))
plt.plot(monthly['month'], monthly['revenue'], 
        marker='o', linewidth=3, markersize=12, color='#2ecc71')
plt.fill_between(monthly['month'], 0, monthly['revenue'], alpha=0.3, color='#2ecc71')
plt.title('Monthly Revenue Trend', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('Revenue (€)', fontsize=12)
plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../outputs/chart2_monthly_trend.png', dpi=150, bbox_inches='tight')
plt.close()
print("✅ Saved: chart2_monthly_trend.png")

# -------------------------
# Chart 3: Segment Pie
# -------------------------
plt.figure(figsize=(10, 8))
segments_sorted = segments.sort_values('revenue', ascending=False)
plt.pie(segments_sorted['revenue'], labels=segments_sorted['market_segment'],
       autopct='%1.1f%%', startangle=90, 
       colors=plt.cm.Set3(range(len(segments_sorted))))
plt.title('Revenue Share by Market Segment', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../outputs/chart3_segment_pie.png', dpi=150, bbox_inches='tight')
plt.close()
print("✅ Saved: chart3_segment_pie.png")

# -------------------------
# Chart 4: Weekend Comparison
# -------------------------
plt.figure(figsize=(10, 6))
categories = ['Bookings', 'Revenue', 'ADR']
weekday_vals = [weekday_row['bookings'], weekday_row['revenue'], weekday_row['avg_adr']]
weekend_vals = [weekend_row['bookings'], weekend_row['revenue'], weekend_row['avg_adr']]
x = [0, 1, 2]
width = 0.35
plt.bar([i - width/2 for i in x], weekday_vals, width, 
       label='Weekday', color='#3498db', alpha=0.8)
plt.bar([i + width/2 for i in x], weekend_vals, width, 
       label='Weekend', color='#e74c3c', alpha=0.8)
plt.title('Weekend vs Weekday Performance', fontsize=16, fontweight='bold')
plt.ylabel('Value', fontsize=12)
plt.xticks(x, categories)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('../outputs/chart4_weekend_comparison.png', dpi=150, bbox_inches='tight')
plt.close()
print("✅ Saved: chart4_weekend_comparison.png")

print(f"\n✅ All 4 charts saved to: ../outputs/")
print("   You can view them as image files!")
print("   These are perfect for your portfolio and presentations!")


💾 Saving charts as image files...
✅ Saved: chart1_revenue_by_room.png
✅ Saved: chart2_monthly_trend.png
✅ Saved: chart3_segment_pie.png
✅ Saved: chart4_weekend_comparison.png

✅ All 4 charts saved to: ../outputs/
   You can view them as image files!
   These are perfect for your portfolio and presentations!


In [41]:
# Add this as a NEW CELL at the very end:

import sqlite3
import pandas as pd

# Reconnect to database
conn = sqlite3.connect('../data/hotel_revenue.db')

# Then paste the rest of the export code from the guide
# (Copy from the CORRECTED_Export_Code.md file)

In [44]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('../data/hotel_revenue.db')

# Fixed query with correct column names
query = """
SELECT 
    f.booking_key,
    d.date as arrival_date,
    d.year, d.quarter, d.month, d.month_name, d.is_weekend,
    r.room_type,
    s.market_segment,
    f.is_cancelled, f.nights, f.guests, f.adr, f.revenue
FROM fact_bookings f
JOIN dim_date d ON f.date_key = d.date_key
JOIN dim_room r ON f.room_key = r.room_key
JOIN dim_segment s ON f.segment_key = s.segment_key
"""

bookings_export = pd.read_sql(query, conn)
bookings_export.to_csv('../data/powerbi_bookings.csv', index=False)
print(f"✅ Bookings: {len(bookings_export):,} rows")

# Monthly aggregates (same query works)
query_monthly = """
SELECT 
    d.year, d.month, d.month_name, d.quarter,
    COUNT(f.booking_key) as total_bookings,
    SUM(CASE WHEN f.is_cancelled = 0 THEN 1 ELSE 0 END) as completed_bookings,
    SUM(CASE WHEN f.is_cancelled = 1 THEN 1 ELSE 0 END) as cancelled_bookings,
    ROUND(SUM(CASE WHEN f.is_cancelled = 0 THEN f.revenue ELSE 0 END), 2) as revenue,
    ROUND(AVG(CASE WHEN f.is_cancelled = 0 THEN f.adr END), 2) as avg_adr,
    ROUND(AVG(CASE WHEN f.is_cancelled = 0 THEN f.nights END), 2) as avg_los
FROM fact_bookings f
JOIN dim_date d ON f.date_key = d.date_key
GROUP BY d.year, d.month, d.month_name, d.quarter
ORDER BY d.year, d.month
"""

monthly_export = pd.read_sql(query_monthly, conn)
monthly_export.to_csv('../data/powerbi_monthly.csv', index=False)
print(f"✅ Monthly: {len(monthly_export)} rows")

# Room performance
query_rooms = """
SELECT 
    r.room_type,
    COUNT(f.booking_key) as bookings,
    ROUND(SUM(CASE WHEN f.is_cancelled = 0 THEN f.revenue ELSE 0 END), 2) as revenue,
    ROUND(AVG(CASE WHEN f.is_cancelled = 0 THEN f.adr END), 2) as avg_adr,
    ROUND(AVG(CASE WHEN f.is_cancelled = 0 THEN f.nights END), 2) as avg_nights
FROM fact_bookings f
JOIN dim_room r ON f.room_key = r.room_key
WHERE f.is_cancelled = 0
GROUP BY r.room_type
ORDER BY revenue DESC
"""

rooms_export = pd.read_sql(query_rooms, conn)
rooms_export.to_csv('../data/powerbi_rooms.csv', index=False)
print(f"✅ Rooms: {len(rooms_export)} rows")

conn.close()
print("\n🎉 ALL DATA EXPORTED!")

✅ Bookings: 97,391 rows
✅ Monthly: 20 rows
✅ Rooms: 8 rows

🎉 ALL DATA EXPORTED!
